In [1]:
pip install tensorflow keras


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load the VGG16 model pre-trained on ImageNet, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
base_model.trainable = False

# Build the model
model = models.Sequential()

# Add the VGG16 base model
model.add(base_model)

# Add new top layers
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Define the paths to your dataset
dataset_path = 'C:/Users/patha/Desktop/SCIT_DSDA_Sem3/Fraud Analytics/Project/data/CASIA2'

# Create an ImageDataGenerator with a split for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalizing the images
    shear_range=0.2,          # Data augmentation (optional)
    zoom_range=0.2,           # Data augmentation (optional)
    horizontal_flip=True,      # Data augmentation (optional)
    validation_split=0.2      # 80/20 split for training/validation
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator (80% of the data)
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',      # For binary classification (fake/real)
    subset='training',        # This tells it to use 80% of the data for training
    shuffle=True              # Shuffle the data for training
)

# Validation data generator (20% of the data)
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',      # For binary classification
    subset='validation',      # This tells it to use 20% of the data for validation
    shuffle=False             # No need to shuffle validation data
)


# # Data augmentation
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    shear_range=0.2,
#                                    zoom_range=0.2,
#                                    horizontal_flip=True)

# test_datagen = ImageDataGenerator(rescale=1./255)

# # Load the dataset
# train_generator = train_datagen.flow_from_directory(
#     'C:/Users/patha/Desktop/SCIT_DSDA_Sem3/Fraud Analytics/Project/data/train',
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='binary')

# validation_generator = test_datagen.flow_from_directory(
#     'C:/Users/patha/Desktop/SCIT_DSDA_Sem3/Fraud Analytics/Project/data/test',
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='binary')

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Adjust according to dataset size
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50)  # Adjust according to dataset size


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21137729 (80.63 MB)
Trainable params: 6423041 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
Found 100

In [2]:
# Save the entire model
model.save('vgg16_casia_model2.h5')


C:\Users\patha\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
from tensorflow.keras.models import load_model

# Load the entire model
model = load_model('vgg16_casia_model.h5')

# Now the model is ready for inference or further training


In [11]:
# Assuming you have new images to predict
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess the image
img = image.load_img("C:/Users/patha/Desktop/SCIT_DSDA_Sem3/Fraud Analytics/Project/data/train/REAL/0000 (2).jpg", target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalizing the image as done during training

# Make a prediction
prediction = model.predict(img_array)

# Apply threshold
threshold = 0.5
if prediction <= threshold:
    print("Fake")
else:
    print("Real")


# Output prediction
print(f'Prediction: {prediction}')


1/1 [==============================] - 0s 288ms/step
Real
Prediction: [[0.88319254]]


In [12]:
# Assuming you have new images to predict
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess the image
img = image.load_img("C:/Users/patha/Desktop/SCIT_DSDA_Sem3/Fraud Analytics/Project/data/train/FAKE/1000 (6).jpg", target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalizing the image as done during training

# Make a prediction
prediction = model.predict(img_array)

# Apply threshold
threshold = 0.5
if prediction <= threshold:
    print("Fake")
else:
    print("Real")


# Output prediction
print(f'Prediction: {prediction}')


1/1 [==============================] - 0s 281ms/step
Fake
Prediction: [[0.02289654]]


In [13]:
import exifread

def extract_metadata(image_path):
    with open(image_path, 'rb') as image_file:
        tags = exifread.process_file(image_file)
    metadata = {
        'DateTime': tags.get('EXIF DateTimeOriginal', 'N/A'),
        'Software': tags.get('Image Software', 'N/A')
    }
    return metadata
